In [4]:
import pandas as pd
import glob
from tqdm import tqdm

In [11]:
df = pd.read_csv("/data/ephemeral/home/yujin/filtered_df.csv")

In [38]:
vocab_files = {
    0: "/data/ephemeral/home/yujin/data_labeling/final_target_0_vocab.csv",
    1: "/data/ephemeral/home/yujin/data_labeling/final_target_1_vocab.csv",
    2: "/data/ephemeral/home/yujin/data_labeling/final_target_2_vocab.csv",
    3: "/data/ephemeral/home/yujin/data_labeling/final_target_3_vocab.csv",
    4: "/data/ephemeral/home/yujin/data_labeling/final_target_4_vocab.csv",
    5: "/data/ephemeral/home/yujin/data_labeling/final_target_5_vocab.csv",
    6: "/data/ephemeral/home/yujin/data_labeling/final_target_6_vocab.csv",
}

In [39]:
target_vocab = {}

for target, file_path in vocab_files.items():
    vocab_df = pd.read_csv(file_path)
    target_vocab[target] = vocab_df['word'].tolist()

print(target_vocab)

{0: ['해안', '축제', '여행', '날씨', '한국', '신간', '제주', '남쪽', '지역', '예술', '구름', '시간', '전국', '서울', '공연', '주말', '주의보', '열리다', '연주', '세상', '즐기다', '영화', '개최', '동안', '먼지', '위하다', '걷기', '미세', '충북', '퍼센트', '개막', '인기', '시작', '국제', '영하', '관광객', '문화', '배우다', '강원', '작가', '세기', '배우', '오후', '작품', '내일', '아침', '베스트셀러', '행사', '제주도', '기온', '사진', '한파', '내리다', '이야기', '부산', '올림픽', '소설', '지원', '운동', '문학', '진행', '발령', '자전거', '경북', '콘서트', '사랑', '피아노', '클래식', '영향', '바람', '떨어지다', '출간', '현대', '미술', '오전', '아트', '무료', '수상', '박물관', '경기', '힐링', '과학', '가을', '스타', '사람', '북쪽', '주년', '미래', '겨울', '사우디아라비아', '예상', '얼음', '증가', '전북', '인도', '일본', '최고', '북한', '그림', '에세이', '떠나다', '국내', '지방', '공개', '소식', '산천어', '전통', '기록', '이상', '온도', '시민', '해제', '만화', '할인', '이전', '뉴스', '고속', '게시판', '대하다', '영화제', '올해', '보내다', '독립', '한류', '기념', '소리', '우주', '그치다', '장마', '강수량', '세계', '장례식', '관심', '관찰', '영상', '보도', '지속', '레스토랑', '연극', '현실', '하루', '여성', '마지막', '도전', '밀리미터', '중부', '만나다', '트렌드', '감사', '특별', '역사', '거리', '오존', '가족', '경보', '생각', '모습', '평화', '새해'

In [40]:
def find_mislabeled_data(df, text_col, target_col):
    """
    잘못된 라벨을 가진 데이터를 찾는 함수
    
    Args:
        df: 데이터프레임 (train dataset)
        text_col: 텍스트 열 이름 (예: 'text')
        target_col: 라벨 열 이름 (예: 'target')
    
    Returns:
        mislabeled_df: 잘못된 라벨을 가진 데이터를 포함한 데이터프레임
    """
    mislabeled_data = []

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        text = row[text_col]
        true_label = row[target_col]

        # 각 라벨별로 해당 라벨의 단어 리스트에 포함된 단어가 텍스트에 있는지 확인
        for label, vocab_list in target_vocab.items():
            if any(word in text for word in vocab_list):
                # 라벨이 해당 단어 리스트의 라벨과 일치하지 않으면 해당 데이터 저장
                if true_label != label:
                    mislabeled_data.append(row)
                break  # 일치하는 라벨을 찾으면 더 이상 탐색하지 않음

    # 잘못된 라벨을 가진 데이터프레임 생성
    mislabeled_df = pd.DataFrame(mislabeled_data)
    
    return mislabeled_df

In [41]:
# 잘못된 라벨을 가진 데이터 찾기
mislabeled_df = find_mislabeled_data(df, text_col='text', target_col='target')
mislabeled_df

100%|██████████| 1203/1203 [00:00<00:00, 11380.47it/s]


,ID,text,target,is_noise_1
0,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,False
2,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합,6,False
3,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감,4,False
4,ynat-v1_train_00011,NH투자 1월 옵션 만기일 매도 우세,1,False
5,ynat-v1_train_00012,황총리 각 부처 비상대비태세 철저히 강구해야,2,False
...,...,...,...,...
1197,ynat-v1_train_02793,경찰 5월초 유커와 日관광객위해 바가지 요금 집중 단속,1,False
1198,ynat-v1_train_02794,문 대통령 김기식 금감원장 사표 수리키로종합,2,False
1199,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6,False
1200,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2,False
